In [1]:
import sys
sys.path.append("/data/Beatriz/Doctorado GR/ADL_platform//S-ADL")

In [2]:
from SADL.time_series.time_series_utils import TimeSeriesProcessor, TimeSeriesDatasetV2
from SADL.time_series.algorithms import tsfedl
from SADL.time_series.preprocessing.preprocessing_ts import StandardScalerPreprocessing 
from SADL.time_series.time_series_datasets_uci import global_load as load_time_series 
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, TensorDataset
from SADL.visualization_module import DataVisualization

2025-09-25 13:03:37.958091: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-25 13:03:37.970318: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758798217.984440  133288 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758798217.988219  133288 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-25 13:03:38.003030: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

#### Example with ai4i_2020_predictive_maintenance_dataset 

##### Load and preprocess datasets

In [3]:
X,y = load_time_series('ai4i_2020_predictive_maintenance_dataset')   #name dataset in static datasets uci repo
labels = y["Machine failure"]
X = X.drop('Type',axis=1)  # remove Type or apply one hot encoding

Metadata: {'uci_id': 601, 'name': 'AI4I 2020 Predictive Maintenance Dataset', 'repository_url': 'https://archive.ics.uci.edu/dataset/601/ai4i+2020+predictive+maintenance+dataset', 'data_url': 'https://archive.ics.uci.edu/static/public/601/data.csv', 'abstract': 'The AI4I 2020 Predictive Maintenance Dataset is a synthetic dataset that reflects real predictive maintenance data encountered in industry.', 'area': 'Computer Science', 'tasks': ['Classification', 'Regression', 'Causal-Discovery'], 'characteristics': ['Multivariate', 'Time-Series'], 'num_instances': 10000, 'num_features': 6, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['Machine failure', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF'], 'index_col': ['UID', 'Product ID'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2020, 'last_updated': 'Wed Feb 14 2024', 'dataset_doi': '10.24432/C5HS5C', 'creators': [], 'intro_paper': {'ID': 386, 'type': 'NATIVE', 'title': 'Explainable Artificial 

In [4]:
scaler = StandardScalerPreprocessing()
X_scaled = scaler.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X_scaled, labels, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape )

(8000, 5) (2000, 5) (8000,) (2000,)


Time Series Dataset

In [8]:
wsize = 24
step = 1
n_pred = 1
processor = TimeSeriesProcessor(window_size= wsize, step_size=1, future_prediction=False, n_pred=n_pred)
X_train_windows, y_train_windows, X_test_windows, y_test_windows = processor.process_train_test(X_train, y_train, X_test, y_test)
print("X_train shape:", X_train_windows.shape)
print("y_train shape:", y_train_windows.shape)
print("X_test shape:", X_test_windows.shape)
print("y_test shape:", y_test_windows.shape)

X_train shape: (7977, 24, 5)
y_train shape: (7977, 24)
X_test shape: (1977, 24, 5)
y_test shape: (1977, 24)


In [9]:

X_train_windows = torch.tensor(X_train_windows, dtype=torch.float32)
y_train_windows = torch.tensor(y_train_windows, dtype=torch.float32)

y_train_windows = y_train_windows.unsqueeze(-1)  # to (7977, 24, 1)


In [7]:
# train_dataset = TensorDataset(X_train_windows, y_train_windows)
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=3, shuffle=True)

##### Define model ohshulih

In [10]:
from TSFEDL.models_pytorch import OhShuLih_Forecaster
top_module = OhShuLih_Forecaster(out_features=5,n_pred=1)
top_module.in_features

20

In [11]:

kwargs = {"algorithm_": "ohshulih", "loss": torch.nn.MSELoss(),"top_module": top_module, "max_epochs": 1, "in_features":24}
model1 = tsfedl.TsfedlAnomalyDetection(**kwargs)
#model1.model = model1.model.double()


{'label_parser': None, 'algorithm_': 'OhShuLih', 'pytorch_params_': {}, 'in_features': 24, 'loss': CrossEntropyLoss(), 'metrics': None, 'optimizer': None, 'top_module': None}


In [13]:
model1.fit(X_train_windows,y_train_windows)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


/data/Beatriz/Doctorado GR/ADL_platform/S-ADL/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.

  | Name         | Type                | Params | Mode 
-------------------------------------------------------------
0 | loss         | MSELoss             | 0      | train
1 | classifier   | OhShuLih_Forecaster | 105    | train
2 | convolutions | Sequential          | 1.8 K  | train
3 | lstm         | LSTM                | 2.2 K  | train
-------------------------------------------------------------
4.1 K     Trainable params
0         Non-trainable params
4.1 K     Total params
0.017     Total estimated model params size (MB)
14        Modules in train mode
0         Modules in eval mode
/data/Beatriz/Doctorado GR/ADL_platform/S-ADL/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many wo

Training: |          | 0/? [00:00<?, ?it/s]

/data/Beatriz/Doctorado GR/ADL_platform/S-ADL/venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([16, 24, 1])) that is different to the input size (torch.Size([16, 1, 5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/data/Beatriz/Doctorado GR/ADL_platform/S-ADL/venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([9, 24, 1])) that is different to the input size (torch.Size([9, 1, 5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
`Trainer.fit` stopped: `max_epochs=1` reached.


In [11]:
scores_pred = model1.decision_function(X_test_windows)
print("Scores",scores_pred)


Scores [[ 0.44721     1.0354289   0.6125488  ...  0.8897025   1.2331866
   0.5473066 ]
 [ 1.032369    0.61542386  1.2181275  ...  1.229087    0.54725873
   1.8020117 ]
 [ 0.6065802   1.2200361   1.482495   ...  0.54831475  1.8164831
   0.7013031 ]
 ...
 [ 0.7597419   1.9198883   1.1575725  ...  0.4408958   1.1302881
   1.6254857 ]
 [ 1.9012139   1.1706023   0.81549513 ...  1.1355283   1.623468
  12.061449  ]
 [ 1.1656959   0.8199609   2.0842588  ...  1.6258647  12.054708
   1.5709175 ]]


In [12]:
y_pred = model1.predict(X_test_windows)
y_pred.shape

(1977, 24)

In [13]:
import numpy as np
y_pred =np.array(y_pred).ravel()
y_true= np.array(y_test_windows).ravel()

In [14]:
y_pred.shape

(47448,)

In [15]:
y_true.shape

(47448,)

In [16]:
y_pred == y_true

array([ True, False,  True, ...,  True,  True,  True])

In [17]:
visualizer = DataVisualization(data=X_test, plot_technique='plot_anomaly', dim_reduction_technique='PCA', y_true=y_true, y_pred=y_pred,subset_size_percent=0.2)
visualizer.fit()
visualizer.show()

#### Example with metro_interstate_traffic_volume dataset

In [3]:
X,y = load_time_series('metro_interstate_traffic_volume')
labels = y["traffic_volume"]
X = X.drop(["date_time", "holiday","weather_main","weather_description"], axis=1)

Metadata: {'uci_id': 492, 'name': 'Metro Interstate Traffic Volume', 'repository_url': 'https://archive.ics.uci.edu/dataset/492/metro+interstate+traffic+volume', 'data_url': 'https://archive.ics.uci.edu/static/public/492/data.csv', 'abstract': 'Hourly Minneapolis-St Paul, MN traffic volume for westbound I-94. Includes weather and holiday features from 2012-2018.', 'area': 'Other', 'tasks': ['Regression'], 'characteristics': ['Multivariate', 'Sequential', 'Time-Series'], 'num_instances': 48204, 'num_features': 8, 'feature_types': ['Integer', 'Real'], 'demographics': [], 'target_col': ['traffic_volume'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2019, 'last_updated': 'Fri Mar 15 2024', 'dataset_doi': '10.24432/C5X60B', 'creators': ['John Hogue'], 'intro_paper': None, 'additional_info': {'summary': 'Hourly Interstate 94 Westbound traffic volume for MN DoT ATR station 301, roughly midway between Minneapolis and St Paul, MN. Ho

In [5]:
scaler = StandardScalerPreprocessing()
X_scaled = scaler.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X_scaled, labels, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape )

(38563, 4) (9641, 4) (38563,) (9641,)


In [6]:
wsize = 24
step = 1
n_pred = 1
processor = TimeSeriesProcessor(window_size= wsize, step_size=1, future_prediction=False, n_pred=n_pred)
X_train_windows, y_train_windows, X_test_windows, y_test_windows = processor.process_train_test(X_train, y_train, X_test, y_test)
print("X_train shape:", X_train_windows.shape)
print("y_train shape:", y_train_windows.shape)
print("X_test shape:", X_test_windows.shape)
print("y_test shape:", y_test_windows.shape)

X_train shape: (38540, 24, 4)
y_train shape: (38540, 24)
X_test shape: (9618, 24, 4)
y_test shape: (9618, 24)


In [7]:
X_train_windows = torch.tensor(X_train_windows, dtype=torch.float32)
y_train_windows = torch.tensor(y_train_windows, dtype=torch.float32)

y_train_windows = y_train_windows.unsqueeze(-1)  # to (7977, 24, 1)


In [8]:
from TSFEDL.models_pytorch import OhShuLih_Forecaster
top_module = OhShuLih_Forecaster(out_features=4,n_pred=1)
top_module.in_features

20

In [9]:

kwargs = {"algorithm_": "ohshulih", "batch_size":64,"loss": torch.nn.MSELoss(),"top_module": top_module, "max_epochs": 1, "in_features":24}
model1 = tsfedl.TsfedlAnomalyDetection(**kwargs)
#model1.model = model1.model.double()


{'label_parser': None, 'algorithm_': 'OhShuLih', 'pytorch_params_': {}, 'in_features': 24, 'loss': CrossEntropyLoss(), 'metrics': None, 'optimizer': None, 'top_module': None}


In [11]:
model1.fit(X_train_windows,X_train_windows)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/data/Beatriz/Doctorado GR/ADL_platform/S-ADL/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.

  | Name         | Type                | Params | Mode 
-------------------------------------------------------------
0 | loss         | MSELoss             | 0      | train
1 | classifier   | OhShuLih_Forecaster | 84     | train
2 | convolutions | Sequential          | 1.8 K  | train
3 | lstm         | LSTM                | 2.2 K  | train
-------------------------------------------------------------
4.1 K     Trainable params
0         Non-trainable params
4.1 K     Total params
0.016     Total estimated model params size (MB)
14        Modules in train mode
0         Modules in eval mode
/data/Beatriz/Doctorado GR/ADL_platform/S-ADL/venv/lib/python3.10/site-pa

Training: |          | 0/? [00:00<?, ?it/s]

/data/Beatriz/Doctorado GR/ADL_platform/S-ADL/venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([64, 24, 4])) that is different to the input size (torch.Size([64, 1, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/data/Beatriz/Doctorado GR/ADL_platform/S-ADL/venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([12, 24, 4])) that is different to the input size (torch.Size([12, 1, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
`Trainer.fit` stopped: `max_epochs=1` reached.


In [11]:
scores = model1.decision_function(X_test_windows)
pred = model1.predict(X_test_windows)

print("Anomaly scores:", scores.shape)
print("Predicted labels:", pred.shape)

Anomaly scores: (9618,)
Predicted labels: (9618, 4)


In [29]:
r = model1.evaluate(X_test_windows)

In [40]:
# visualizer = DataVisualization(data=X_test, plot_technique='', dim_reduction_technique='PCA', y_true=labels_true, y_pred=labels_pred,subset_size_percent=0.2)
# visualizer.fit()
# visualizer.show()